# Урок 2. Парсинг HTML. BeautifulSoup, MongoDB

1) Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайта superjob.ru и hh.ru. Приложение должно анализировать несколько страниц сайта(также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:

    *Наименование вакансии
    *Предлагаемую зарплату (отдельно мин. и отдельно макс. и отдельно валюта)
    *Ссылку на саму вакансию        
    *Сайт откуда собрана вакансия
По своему желанию можно добавить еще работодателя и расположение. Данная структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas.

In [281]:
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd

In [282]:
def parser_hh(soup):
    vacancy_block = soup.find('div',{'class':'vacancy-serp'})
    vacancy_list = vacancy_block.find_all('div',{'class':'vacancy-serp-item'})
    vacancies = []
    for vacancy in vacancy_list:
        vacancy_data = {}
        vacancy_name = vacancy.find('a', {'class':'bloko-link'}).getText()
        vacancy_link = vacancy.find('a',{'class':'bloko-link'})['href']
        vacancy_salary = vacancy.find('div', {'class':'vacancy-serp-item__sidebar'}).getText()
        if not vacancy_salary:
            vacancy_salary_min = None
            vacancy_salary_max = None
            vacancy_salary_currency = None
        else:
            vacancy_salary = vacancy_salary \
                            .replace(u'\xa0', u'')

            vacancy_salary = re.split(r'\s|-', vacancy_salary)

            if vacancy_salary[0] == 'до':
                vacancy_salary_min = None
                vacancy_salary_max = int(vacancy_salary[1])
            elif vacancy_salary[0] == 'от':
                vacancy_salary_min = int(vacancy_salary[1])
                vacancy_salary_max = None
            else:
                vacancy_salary_min = int(vacancy_salary[0])
                vacancy_salary_max = int(vacancy_salary[1])            

            vacancy_salary_currency = vacancy_salary[2]

        vacancy_data['name'] = vacancy_name
        vacancy_data['link'] = vacancy_link
        vacancy_data['salary_min'] = vacancy_salary_min
        vacancy_data['salary_max'] = vacancy_salary_max
        vacancy_data['salary_currency'] = vacancy_salary_currency
        vacancy_data['site'] = 'hh.ru'

        vacancies.append(vacancy_data)
    return vacancies

In [283]:
hh_main_link = 'https://hh.ru'
query = 'Python'
hh_page = 0
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:69.0) Gecko/20100101 Firefox/69.0'}

hh_vacancy_date = []

while True:
    hh_response = requests.get(hh_main_link + '/search/vacancy?text=' + query + '&page=' + str(hh_page), headers=headers).text
    hh_soup = bs(hh_response,'lxml')
    hh_vacancy_date = hh_vacancy_date + parser_hh(hh_soup)
    hh_page += 1
    if hh_soup.find('a',{'class':'HH-Pager-Controls-Next'}) is None:
        break

In [342]:
def parser_sj(soup, main_link):
#     vacancy_block = soup.find('div',{'class':'display: block;'})
#     vacancy_list = vacancy_block.find_all('div',{'class':'f-test-vacancy-item'})
    vacancy_list = soup.find_all('div',{'class':'f-test-vacancy-item'})
    vacancies = []
    for vacancy in vacancy_list:
        vacancy_data = {}
        vacancy_name = vacancy.find('a', {'class':'icMQ_'}).getText()
        vacancy_link = main_link + vacancy.find('a',{'class':'icMQ_'})['href']    
        vacancy_salary = vacancy.find('span', {'class':'f-test-text-company-item-salary'}).findChildren()
        if not vacancy_salary or vacancy_salary[-1].getText() == 'По договорённости':
            vacancy_salary_min = None
            vacancy_salary_max = None
            vacancy_salary_currency = None
        else:
            vacancy_salary_currency = vacancy_salary[-4].getText().replace(u'\xa0', u' ').split(' ')[-1]
            is_check_sarary = vacancy.find('span', {'class': 'f-test-text-company-item-salary'}) \
                                                .getText() \
                                                .replace(u'\xa0', u' ') \
                                                .split(' ', 1)[0]
            if is_check_sarary == 'до' or len(vacancy_salary) == 2:
                vacancy_salary_min = None
                vacancy_salary_max = (vacancy_salary[0].getText().replace(u'\xa0', u''))
                vacancy_salary_max = int(re.search('(?<=до).*?(?!\d)', vacancy_salary_max)[0])
            elif is_check_sarary == 'от':
                vacancy_salary_min = (vacancy_salary[0].getText().replace(u'\xa0', u''))
                vacancy_salary_min = int(re.search('(?<=от).*?(?!\d)', vacancy_salary_min)[0])
                vacancy_salary_max = None
            else:
                is_check_sarary1 = vacancy.find('span', {'class': 'f-test-text-company-item-salary'}) \
                                                .getText() \
                                                .replace(u'\xa0', u' ') \
                                                .split(' ')[2]
                if is_check_sarary1 == '—':  
                    vacancy_salary_min = (vacancy_salary[0].getText().replace(u'\xa0', u''))
                    vacancy_salary_min = int(re.search('.+?(?=—)', vacancy_salary_min)[0])
                
                    vacancy_salary_max = (vacancy_salary[0].getText().replace(u'\xa0', u'')) 
                    vacancy_salary_max = int(re.search('(?<=—).+?(?!\d)', vacancy_salary_max)[0])
                else:
                    vacancy_salary_min = (vacancy_salary[0].getText().replace(u'\xa0', u''))
                    vacancy_salary_min = int(re.search('.*?(?!\d)', vacancy_salary_min)[0])
                
                    vacancy_salary_max = vacancy_salary_min 
                            
        vacancy_data['name'] = vacancy_name
        vacancy_data['link'] = vacancy_link
        vacancy_data['salary_min'] = vacancy_salary_min
        vacancy_data['salary_max'] = vacancy_salary_max
        vacancy_data['salary_currency'] = vacancy_salary_currency
        vacancy_data['site'] = 'superjob.ru'

        vacancies.append(vacancy_data)
    return vacancies

In [343]:
sj_main_link = 'https://www.superjob.ru'
query = 'Python'
sj_page = 1
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:69.0) Gecko/20100101 Firefox/69.0'}

sj_vacancy_date = []


while True:
    sj_response = requests.get(sj_main_link + '/vacancy/search/?keywords=' + query + '&page=' + str(sj_page), headers=headers).text
    sj_soup = bs(sj_response,'lxml')
    
    sj_vacancy_date = sj_vacancy_date + parser_sj(sj_soup, sj_main_link)
    sj_page += 1
    if sj_soup.find('a',{'class':'f-test-button-dalshe'}) is None:
        break

In [344]:
vacancy_date = []
vacancy_date.extend(sj_vacancy_date)
vacancy_date.extend(hh_vacancy_date)
    
df = pd.DataFrame(vacancy_date)

In [346]:
df[50:80]

,name,link,salary_min,salary_max,salary_currency,site
50,Web-разработчик Full stack,https://www.superjob.ru/vakansii/web-razrabotc...,NaN,200000.0,руб.,superjob.ru
51,Team lead / Ведущий разработчик,https://www.superjob.ru/vakansii/team-lead-339...,NaN,300000.0,руб.,superjob.ru
52,Reverse Engineer / Вирусный аналитик,https://www.superjob.ru/vakansii/reverse-engin...,NaN,300000.0,руб.,superjob.ru
53,Программист микроконтроллеров,https://www.superjob.ru/vakansii/programmist-m...,NaN,NaN,None,superjob.ru
54,ML/NLP Architect / ML/NLP Архитектор,https://www.superjob.ru/vakansii/ml-33788781.html,120000.0,NaN,руб.,superjob.ru
55,NLP Engineer / NLP инженер,https://www.superjob.ru/vakansii/nlp-engineer-...,120000.0,NaN,руб.,superjob.ru
56,Программист-разработчик python (midle),https://khimki.hh.ru/vacancy/37674249?query=Py...,100000.0,180000.0,руб.,hh.ru
57,Python Developer,https://khimki.hh.ru/vacancy/37279680?query=Py...,NaN,NaN,None,hh.ru
58,Data Scientist,https://khimki.hh.ru/vacancy/37645302?query=Py...,NaN,NaN,None,hh.ru
59,PHP программист,https://khimki.hh.ru/vacancy/37678834?query=Py...,500000.0,700000.0,KZT,hh.ru
